## Сетевое программирование в Python

- сокеты
- CGI
- requests + BeautifulSoup
- aiohttp
- websockets
- flask
- scrapy

### Сокеты

In [1]:
with open('script/socket_server.py', 'rt') as f:
    print(f.read())


import socket
import time

# create a socket object
serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# get local machine name
host = socket.gethostname()

port = 9999

# bind to the port
serversocket.bind((host, port))

# queue up to 5 requests
serversocket.listen(5)

while True:
    # establish a connection
    clientsocket,addr = serversocket.accept()

    print("Got a connection from %s" % str(addr))
    currentTime = time.ctime(time.time()) + "\r\n"
    clientsocket.send(currentTime.encode('ascii'))
    clientsocket.close()



In [2]:
with open('script/socket_client.py', 'rt') as f:
    print(f.read())


import socket

HOST = socket.gethostname()    
PORT = 9999

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((HOST, PORT))
sock.sendall(b'Hello, world')

data = sock.recv(1024)

sock.close()

print('Received', repr(data))



In [3]:
# Запускаем:

# > python socket_server.py
# > python socket_client.py

# ответ на клиенте будет в виде:
# Received b'Tue May  4 23:17:19 2021\r\n'


In [4]:
print('Пример эхо-сервера:')
print()
print('===')

with open('script/echo_server.py', 'rt') as f:
    print(f.read())

Пример эхо-сервера:

===
import socket

HOST = ''        # Symbolic name meaning all available interfaces
PORT = 9999

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

sock.bind((HOST, PORT))
sock.listen(1)
conn, addr = sock.accept()

print('Connected by', addr)

while True:
    data = conn.recv(1024)
    #if not data: break
    if data:
        conn.sendall(data + b' from server!')

conn.close()



In [5]:
# Запускаем:

# > python socket_server.py
# > python socket_client.py

# ответ на клиенте будет в виде:
# Received b'Hello, world from server!'


### CGI (Common Gateway Interface)

CGI является средством динамической генерации web-страниц. 

Для написания CGI скриптов пригоден любой интерпретируемый или компилируемый язык программирования. Типичная CGI программа состоит из двух частей: из передачи HTTP заголовков и передачи HTML данных. Веб-сервер связывает вывод CGI скрипта со вводом у браузера. Обратная связь осуществляется передачей данных от сервера клиенту путём передачи параметров.

Скрипты, к-ые будут далее в ячейках, помещаем в директорию **cgi-bin**.

Запускаем встроенный http-сервер (либо другой)

> <pre>python -m http.server --cgi</pre>


In [6]:
with open('script/cgi-bin/simple.py', 'rt', encoding='utf8') as f:
    print(f.read())


print("Content-type: text/html")
print()
print("<h1>Привет</h1>")
print("<h3>от <pre>localhost:8000/cgi-bin/simple.py</pre> ))</h3>")



---

Открываем в браузере url:

> <pre>localhost:8000/cgi-bin/simple.py</pre>


---

Далее рассмотрим пример с передачей данных формы.

Сделаем такую html-страницу:


In [7]:
with open('script/cgi.html', 'rt', encoding='utf8') as f:
    print(f.read())


<!DOCTYPE html>
<html>
<head>
<meta charset='utf-8'>
<title>Форма добавления автомобиля</title>
</head>
  <body>
    <form action='/cgi-bin/add_car.py'>
      Car: 
      <input type='text' name='car'>
      No: 
      <input type='text' name='no'>
      <input type='submit'>
  </form>
</body>
</html>



---

Открываем в браузере url:

> <pre>localhost:8000/cgi.html</pre>

Будет вызываться следующий скрипт:


In [8]:
with open('script/cgi-bin/add_car.py', 'rt', encoding='utf8') as f:
    print(f.read())


import cgi
import html


form = cgi.FieldStorage()
car = html.escape(form.getfirst('car', 'unknown'))
car_no = html.escape(form.getfirst('no', 'unknown'))

print('Content-type: text/html\n')
print('''<!DOCTYPE HTML>
        <html>
        <head>
            <title>Форма добавления автомобиля</title>
        </head>
        <body>''')

print('<h1>Ваши данные приняты!</h1>')
print('<p>Вы указали автомобиль: {}</p>'.format(car))
print('<p>С номером: {}</p>'.format(car_no))

print('''</body></html>''')



### Библиотека requests

In [9]:
import requests

### Библиотека aiohttp

Предоставляет возможности для асинхронного клиент-серверного взаимодействия (http, web sockets).


In [10]:
# !pip install aiohttp

import aiohttp
import asyncio
import time


In [11]:
# example taken from
# https://www.twilio.com/blog/asynchronous-http-requests-in-python-with-aiohttp

start_time = time.time()


async def get_pokemon(session, url):
    async with session.get(url) as resp:
        pokemon = await resp.json()
        return pokemon['name']


async def main():

    async with aiohttp.ClientSession() as session:

        tasks = []
        for number in range(1, 151):
            url = f'https://pokeapi.co/api/v2/pokemon/{number}'
            tasks.append(asyncio.ensure_future(get_pokemon(session, url)))

        original_pokemon = await asyncio.gather(*tasks)
        for pokemon in original_pokemon:
            print(pokemon)

# asyncio.run(main())

await main()

print("--- %s seconds ---" % (time.time() - start_time))


bulbasaur
ivysaur
venusaur
charmander
charmeleon
charizard
squirtle
wartortle
blastoise
caterpie
metapod
butterfree
weedle
kakuna
beedrill
pidgey
pidgeotto
pidgeot
rattata
raticate
spearow
fearow
ekans
arbok
pikachu
raichu
sandshrew
sandslash
nidoran-f
nidorina
nidoqueen
nidoran-m
nidorino
nidoking
clefairy
clefable
vulpix
ninetales
jigglypuff
wigglytuff
zubat
golbat
oddish
gloom
vileplume
paras
parasect
venonat
venomoth
diglett
dugtrio
meowth
persian
psyduck
golduck
mankey
primeape
growlithe
arcanine
poliwag
poliwhirl
poliwrath
abra
kadabra
alakazam
machop
machoke
machamp
bellsprout
weepinbell
victreebel
tentacool
tentacruel
geodude
graveler
golem
ponyta
rapidash
slowpoke
slowbro
magnemite
magneton
farfetchd
doduo
dodrio
seel
dewgong
grimer
muk
shellder
cloyster
gastly
haunter
gengar
onix
drowzee
hypno
krabby
kingler
voltorb
electrode
exeggcute
exeggutor
cubone
marowak
hitmonlee
hitmonchan
lickitung
koffing
weezing
rhyhorn
rhydon
chansey
tangela
kangaskhan
horsea
seadra
goldeen
seakin

In [12]:
# если бы делали это синхронно с помощью requests,
# это заняло бы гораздо больше времени:

start_time = time.time()

for number in range(1, 151):
    url = f'https://pokeapi.co/api/v2/pokemon/{number}'
    resp = requests.get(url)
    pokemon = resp.json()
    print(pokemon['name'])

print("--- %s seconds ---" % (time.time() - start_time))


bulbasaur
ivysaur
venusaur
charmander
charmeleon
charizard
squirtle
wartortle
blastoise
caterpie
metapod
butterfree
weedle
kakuna
beedrill
pidgey
pidgeotto
pidgeot
rattata
raticate
spearow
fearow
ekans
arbok
pikachu
raichu
sandshrew
sandslash
nidoran-f
nidorina
nidoqueen
nidoran-m
nidorino
nidoking
clefairy
clefable
vulpix
ninetales
jigglypuff
wigglytuff
zubat
golbat
oddish
gloom
vileplume
paras
parasect
venonat
venomoth
diglett
dugtrio
meowth
persian
psyduck
golduck
mankey
primeape
growlithe
arcanine
poliwag
poliwhirl
poliwrath
abra
kadabra
alakazam
machop
machoke
machamp
bellsprout
weepinbell
victreebel
tentacool
tentacruel
geodude
graveler
golem
ponyta
rapidash
slowpoke
slowbro
magnemite
magneton
farfetchd
doduo
dodrio
seel
dewgong
grimer
muk
shellder
cloyster
gastly
haunter
gengar
onix
drowzee
hypno
krabby
kingler
voltorb
electrode
exeggcute
exeggutor
cubone
marowak
hitmonlee
hitmonchan
lickitung
koffing
weezing
rhyhorn
rhydon
chansey
tangela
kangaskhan
horsea
seadra
goldeen
seakin

In [13]:
# можем также закинуть синхронные requests.get-ы
# в пул потоков

import concurrent.futures


async def main():

    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:

        loop = asyncio.get_event_loop()
        futures = [
            loop.run_in_executor(
                executor, 
                requests.get, 
                f'https://pokeapi.co/api/v2/pokemon/{i}'
            )
            for i in range(1, 151)
        ]
        for response in await asyncio.gather(*futures):
            print(response.json()['name'])


loop = asyncio.get_event_loop()

start_time = time.time()

# await main()
# loop.run_until_complete(main())

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.0 seconds ---


### WebSockets

In [ ]:
# !pip install websockets

In [14]:
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
import IPython

In [15]:

print('Пример кода сервера:')
print()
print('===')

with open('script/websocket_server.py', 'rt') as f:
    print(f.read())


Пример кода сервера:

===
import asyncio
import websockets


async def hello(websocket, path):
    name = await websocket.recv()
    print(f"< {name}")

    greeting = f"Hello {name}!"

    await websocket.send(greeting)
    print(f"> {greeting}")


start_server = websockets.serve(hello, "localhost", 8765)

asyncio.get_event_loop().run_until_complete(start_server)
asyncio.get_event_loop().run_forever()



In [16]:
# %pycat script/websocket_server.py

# покажем код по-модному, с подсветкой синтаксиса )))

print('Пример кода клиента:')
print()
print('===')

with open('script/websocket_client.py', 'rt') as f:
    code = f.read()
    
formatter = HtmlFormatter()
IPython.display.HTML('<style type="text/css">{}</style>{}'.format(
formatter.get_style_defs('.highlight'),
highlight(code, PythonLexer(), formatter)))


Пример кода клиента:

===


In [17]:
# Запускаем:

# python websocket_server.py
# python websocket_client.py

In [18]:
print('Пример кода сервера с общим состоянием для нескольких клиентов:')
print()
print('===')

with open('script/websocket_state.py', 'rt') as f:
    print(f.read())


Пример кода сервера с общим состоянием для нескольких клиентов:

===
import asyncio
import json
import logging
import websockets

logging.basicConfig()

STATE = {"value": 0}

USERS = set()


def state_event():
    return json.dumps({"type": "state", **STATE})


def users_event():
    return json.dumps({"type": "users", "count": len(USERS)})


async def notify_state():
    if USERS:  # asyncio.wait doesn't accept an empty list
        message = state_event()
        await asyncio.wait([user.send(message) for user in USERS])


async def notify_users():
    if USERS:  # asyncio.wait doesn't accept an empty list
        message = users_event()
        await asyncio.wait([user.send(message) for user in USERS])


async def register(websocket):
    USERS.add(websocket)
    await notify_users()


async def unregister(websocket):
    USERS.remove(websocket)
    await notify_users()


async def counter(websocket, path):
    # register(websocket) sends user_event() to websocket
    await register

---

Запускаем этот скрипт в консоли и открываем в браузере несколько файлов **websockets_demo.html**

In [19]:
with open('script/websockets_demo.html', 'rt') as f:
    code = f.read()
    
start = code.index('<script>') + len('<script>')
end = code.index('</script>')


print('Клиентский javascript-код:')
print()
print('===')

print(code[start:end])

Клиентский javascript-код:

===

            var minus = document.querySelector('.minus'),
                plus = document.querySelector('.plus'),
                value = document.querySelector('.value'),
                users = document.querySelector('.users'),
                websocket = new WebSocket("ws://127.0.0.1:6789/");
            minus.onclick = function (event) {
                websocket.send(JSON.stringify({action: 'minus'}));
            }
            plus.onclick = function (event) {
                websocket.send(JSON.stringify({action: 'plus'}));
            }
            websocket.onmessage = function (event) {
                data = JSON.parse(event.data);
                switch (data.type) {
                    case 'state':
                        value.textContent = data.value;
                        break;
                    case 'users':
                        users.textContent = (
                            data.count.toString() + " user" +
                

## flask

---

Пишем свой сервер:

https://docs.aiohttp.org/en/stable/web_quickstart.html#run-a-simple-web-server